In [1]:
import cv2
import os

File ini diperuntukkan untuk mengambil screenshots dari test_video yang diberikan.

## Cari tahu FPS Video

In [8]:
# Path ke video
video_path = "../source/test_video/OTV1.mp4"  # Ganti dengan path ke video Anda

# Buka video dengan OpenCV
cap = cv2.VideoCapture(video_path)

# Periksa apakah video berhasil dibuka
if not cap.isOpened():
    print("Gagal membuka video.")
else:
    # Dapatkan FPS video
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS video: {fps}")

# Tutup video
cap.release()


FPS video: 29.768096292178853


Kami akan mengambil interval pada 500 ms

Mengapa?
Pada 30 FPS, setiap frame setara dengan:
1 detik = 30 frame → 1 frame = 33 ms.
Interval 500 ms setara dengan mengambil frame setiap 115 frame

Interval ini memastikan bahwa dataset:
Tidak terlalu padat (mengambil semua frame).
Tidak terlalu renggang (kehilangan informasi penting).

## Generate screenshots

In [ ]:
# Path ke folder video
video_folder = "../source/test_video"
output_folder = "../dataset/images_raw"

# Pastikan folder output ada
os.makedirs(output_folder, exist_ok=True)

# Ambil daftar semua video dalam folder
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

# Inisialisasi penomoran gambar
image_count = 0

# Proses setiap video dalam folder
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    
    # Buka video
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frame rate video
    frame_interval = int(fps * 0.5)  # Ambil frame setiap 0.5 detik (1 gambar setiap 15 frame)

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Ambil frame berdasarkan interval
        if frame_count % frame_interval == 0:
            output_path = os.path.join(output_folder, f"image_{image_count:04d}.jpg")
            cv2.imwrite(output_path, frame)
            image_count += 1

        frame_count += 1

    cap.release()

print(f"Proses selesai. Total {image_count} gambar disimpan di {output_folder}.")


Proses selesai. Total 373 gambar disimpan di ../dataset/images.


## Cleaning Label Files
Ada kesalahan dalam anotasi data, jadi perlu dibersihkan (mengganti nilai label)

In [2]:
import os

# Path ke folder label
labels_dir = "../dataset/labels/raw"  # Ganti sesuai path Anda

# Buat mapping angka lama ke angka baru
mapping = {
    "15": "0", "16": "1", "17": "2", "18": "3", "19": "4",
    "20": "5", "21": "6", "22": "7", "23": "8", "24": "9", "25": "10"
}

# Periksa dan perbaiki semua file label
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt"):  # Hanya file .txt
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Ganti angka sesuai mapping
        fixed_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 0 and parts[0] in mapping:  # Ganti hanya jika angka ada dalam mapping
                parts[0] = mapping[parts[0]]
            fixed_lines.append(" ".join(parts) + "\n")

        # Tulis ulang file label
        with open(label_path, "w") as f:
            f.writelines(fixed_lines)

print("Semua file label telah diperbaiki.")


Semua file label telah diperbaiki.


## Membagi Dataset

In [2]:
import os
import random
import shutil

# Path ke folder
images_path = "../dataset/images_raw"
labels_path = "../dataset/labels/raw"
output_images_train = "../dataset/images/train"
output_images_val = "../dataset/images/val"
output_labels_train = "../dataset/labels/train"
output_labels_val = "../dataset/labels/val"


# Buat folder output jika belum ada
os.makedirs(output_images_train, exist_ok=True)
os.makedirs(output_images_val, exist_ok=True)
os.makedirs(output_labels_train, exist_ok=True)
os.makedirs(output_labels_val, exist_ok=True)

# Ambil semua file gambar
image_files = [f for f in os.listdir(images_path) if f.endswith(".jpg")]

# Acak urutan file untuk pembagian
random.shuffle(image_files)

# Bagi dataset (80% train, 20% val)
split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Pindahkan file ke folder train/val
for file in train_files:
    shutil.copy(os.path.join(images_path, file), os.path.join(output_images_train, file))
    label_file = file.replace(".jpg", ".txt")
    if os.path.exists(os.path.join(labels_path, label_file)):
        shutil.copy(os.path.join(labels_path, label_file), os.path.join(output_labels_train, label_file))

for file in val_files:
    shutil.copy(os.path.join(images_path, file), os.path.join(output_images_val, file))
    label_file = file.replace(".jpg", ".txt")
    if os.path.exists(os.path.join(labels_path, label_file)):
        shutil.copy(os.path.join(labels_path, label_file), os.path.join(output_labels_val, label_file))

print("Dataset berhasil dibagi ke folder train dan val.")

Dataset berhasil dibagi ke folder train dan val.


## Revisi Minor
untuk file 249 ke atas, label 0 dan 1 ketuker.

In [1]:
import os

# Path ke folder label
labels_dir = "../dataset/labels/raw"  # Ganti sesuai path Anda

# Buat mapping angka lama ke angka baru
mapping = {
    "0": "1",
    "1": "0"
}

# Periksa dan perbaiki semua file label
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt") and label_file >= "image_0249.txt":  # Filter file >= image_0249
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Ganti angka sesuai mapping
        fixed_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 0 and parts[0] in mapping:  # Ganti hanya jika angka ada dalam mapping
                parts[0] = mapping[parts[0]]
            fixed_lines.append(" ".join(parts) + "\n")

        # Tulis ulang file label
        with open(label_path, "w") as f:
            f.writelines(fixed_lines)

print("Semua file label dari image_0249 ke atas telah diperbaiki.")


Semua file label dari image_0249 ke atas telah diperbaiki.
